In [1]:
import yaml
from pathlib import Path
from strsimpy.cosine import Cosine
from collections import defaultdict
from uuid import uuid4

In [2]:
def get_unique_id():
    return uuid4().hex

In [3]:
def from_yaml(yml_path):
    return yaml.safe_load(yml_path.read_text())

In [4]:
def to_yaml(dict_):
    return yaml.safe_dump(dict_, sort_keys = False, allow_unicode=True)

In [14]:
def get_similarity(title1, title2):
    print(title1, title2)
    cosine = Cosine(2)
    p0 = cosine.get_profile(title1)
    p1 = cosine.get_profile(title2)
    try:
        return cosine.similarity_profiles(p0, p1)
    except:
        print(title1, title2)

In [15]:
def search_text_id(text_titles, title):
    for text_id, text_title in text_titles.items():
        if text_title:
            similarity = get_similarity(text_title, title)
            if similarity > 0.5:
                print(f'{title} => {text_id}')
                return text_id, text_title
    return get_unique_id(),''

In [16]:
def restructure_outline(text_durchen):
    texts = defaultdict(str)
    prev_title = "འདུལ་བ་གཞི།"
    prev_vol = "0"
    cur_text = []
    for text_id, text in text_durchen.items():
        similarity = get_similarity(text["text_title"], prev_title)
        if similarity > 0.8 and text["vol"] != prev_vol:
            cur_text.append(text)
        else:
            texts[cur_text[0]['text_title']] = cur_text
            cur_text = []
            cur_text.append(text)
        prev_title = text["text_title"]
        prev_vol = text["vol"]
    return texts

In [17]:
t2d = Path("./new_pedurma_outline.yml").read_text(encoding="utf-8")
text_durchen = yaml.safe_load(t2d)
outline_texts = restructure_outline(text_durchen)

འདུལ་བ་གཞི འདུལ་བ་གཞི།
འདུལ་བ་གཞི འདུལ་བ་གཞི
འདུལ་བ་གཞི འདུལ་བ་གཞི
འདུལ་བ་གཞི འདུལ་བ་གཞི
སོ་སོར་ཐར་པའི་མདོ འདུལ་བ་གཞི
འདུལ་བ་རྣམ་པར་འབྱེད་པ སོ་སོར་ཐར་པའི་མདོ
འདུལ་བ་རྣམ་པར་འབྱེད་པ འདུལ་བ་རྣམ་པར་འབྱེད་པ
འདུལ་བ་རྣམ་པར་འབྱེད་པ འདུལ་བ་རྣམ་པར་འབྱེད་པ
འདུལ་བ་རྣམ་པར་འབྱེད་པ འདུལ་བ་རྣམ་པར་འབྱེད་པ
དགེ་སློང་མའི་སོ་སོར་ཐར་པའི་མདོ འདུལ་བ་རྣམ་པར་འབྱེད་པ
དགེ་སློང་མའི་འདུལ་བ་རྣམ་པར་འབྱེད་པ དགེ་སློང་མའི་སོ་སོར་ཐར་པའི་མདོ
འདུལ་བ་ཕྲན་ཚེགས་ཀྱི་གཞི དགེ་སློང་མའི་འདུལ་བ་རྣམ་པར་འབྱེད་པ
འདུལ་བ་ཕྲན་ཚེགས་ཀྱི་གཞི འདུལ་བ་ཕྲན་ཚེགས་ཀྱི་གཞི
འདུལ་བ་གཞུང་བླ་མ འདུལ་བ་ཕྲན་ཚེགས་ཀྱི་གཞི
འདུལ་བ་གཞུང་བླ་མ འདུལ་བ་གཞུང་བླ་མ
འདུལ་བ་གཞུང་བླ་མ འདུལ་བ་གཞུང་བླ་མ
འཕགས་པ་བཟང་པོ་སྤྱོད་པའི་སྨོན་ལམ་གྱི་རྒྱལ་པོ འདུལ་བ་གཞུང་བླ་མ
འཕགས་པ་བྱམས་པའི་སྨོན་ལམ འཕགས་པ་བཟང་པོ་སྤྱོད་པའི་སྨོན་ལམ་གྱི་རྒྱལ་པོ
མཆོག་གི་སྤྱོད་པའི་སྨོན་ལམ འཕགས་པ་བྱམས་པའི་སྨོན་ལམ
མར་མེའི་ལོ་རྒྱུས་དང་སྨོན་ལམ་གྱི་མདོ མཆོག་གི་སྤྱོད་པའི་སྨོན་ལམ
འཕགས་པ་གསེར་འོད་དམ་པ་མདོ་སྡེའི་དབང་པོའི་རྒྱལ་པོ་ལས་གསུངས་པའི་སྨོན་ལམ།  མར་མེའི་ལོ་རྒྱུས་དང་སྨོན་ལམ་གྱི་མདོ
སྟོང་ཆེན་མོ་རབ་ཏུ་འཇོམས་པ་ལ

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [90]:
outline_titles = list(outline_texts.keys())

In [91]:
text_titles = from_yaml(Path('./text_title.yml'))

In [92]:
result = {}
for outline_title in outline_titles:
    cur_text = {}
    text_id, text_title = search_text_id(text_titles, outline_title)
    cur_text[text_id] = {
        'text_derge_title': text_title,
        'text_pedurma_title': outline_title,
    }
    if len(text_id)<4:
        del text_titles[text_id]
    result.update(cur_text)
if text_titles:
    for text_id, text_title in text_titles.items():
        cur_text = {}
        cur_text[text_id] = {
            'text_derge_title': text_title,
            'text_pedurma_title': ''
        }
        result.update(cur_text)

འདུལ་བ་གཞི། ༼ཀ༽ => D1
སོར་མདོ། => D2
འདུལ་བ་རྣམ་འབྱེད། ༼ཅ༽ => D3
དགེ་སློང་མའི་སོ་སོར་ཐར་པའི་མདོ། => D4
དགེ་སློང་མའི་འདུལ་བ་རྣམ་པར་འབྱེད་པ། => D5
འདུལ་བ་ཕྲན་ཚེགས་ཀྱི་གཞི། ༼ཐ༽ => D6
འདུལ་བ་གཞུང་བླ་མ། => D7
འདུལ་བ་གཞུང་དམ་པ། ༼བ༽ => D7a
བཟང་སྤྱོད་སྨོན་ལམ། => D1095
བྱམས་སྨོན། => D1096
མཆོག་གི་སྤྱོད་པའི་སྨོན་ལམ། => D336
མར་མེའི་ལོ་རྒྱུས་དང་སྨོན་ལམ་གྱི་མདོ། => D33
གསེར་འོད་དམ་པ་མདོ་སྡེའི་དབང་པོའི་རྒྱལ་པོ་ལས་གསུངས་པའི་སྨོན་ལམ། => D10
སྟོང་ཆེན་མོ་རབ་ཏུ་འཇོམས་པ་ལས་གསུངས་པའི་སྨོན་ལམ། => D11
རིག་སྔགས་ཀྱི་རྒྱལ་མོ་རྨ་བྱ་ཆེན་མོ་ལས་གསུངས་པའི་སྨོན་ལམ་དང་བདེན་ཚིག => D20
ཡངས་པའི་གྲོང་ཁྱེར་དུ་འཇུག་པའི་མདོ་བདེ་ལེགས་ཀྱི་ཚིགས་སུ་བཅད་པ། => D13
བདེ་ལེགས་ཀྱི་ཚིགས་སུ་བཅད་པ། => D73
བདེ་ལེགས་སུ་འགྱུར་བའི་ཚིགས་སུ་བཅད་པ། => D108
ཤིས་པར་བརྗོད་པའི་ཚིགས་སུ་བཅད་པ། => D45
དཀོན་མཆོག་གསུམ་གྱི་བཀྲ་ཤིས་ཀྱི་ཚིགས་སུ་བཅད་པ། => D23
ལྷའི་དབང་པོ་བརྒྱ་བྱིན་ལ་སོགས་པ་གསུམ་གྱིས་བཀྲ་ཤིས་ཚིགས་སུ་བཅད་པ། => D12
ལྷས་ཞུས་པའི་བཀྲ་ཤིས་ཀྱི་ཚིགས་སུ་བཅད་པ། => D14
སངས་རྒྱས་རབས་བདུན་གྱི་བཀྲ་ཤིས་ཀྱི་ཚིགས་སུ་བཅད་པ། => D15
བཀྲ་ཤིས་ཚིགས་སུ་བཅད་པ། => D

མར་མེ་འབུལ་བའི་མདོ། => D204
གྲོང་ཁྱེར་གྱིས་འཚོ་བའི་མདོ། => D205
ཟས་ཀྱི་འཚོ་བ་རྣམ་པར་དག་པའི་མདོ། => D100
གླང་པོའི་རྩལ་གྱི་མདོ། => D102
སྒྲ་ཆེན་པོའི་མདོ། => D107
སེང་གེའི་སྒྲ་བསྒྲགས་པའི་མདོ། => D209
སཱ་ལུའི་ལྗང་པའི་མདོ། => D210
རྟེན་ཅིང་འབྲེལ་བར་འབྱུང་བ་དང་པོ་དང་རྣམ་པར་དབྱེ་བ་བསྟན་པའི་མདོ། => D101
རྟེན་འབྲེལ་ཆེན་པོའི་མདོ། => D111
སོར་མོའི་ཕྲེང་བ་ལ་ཕན་པའི་མདོ། => D207
རྒྱལ་པོ་ལ་གདམས་པའི་མདོ། => D100
མ་སྐྱེས་དགྲའི་འགྱོད་པ་བསལ་བའི་མདོ། => D100
དཔལ་སྦས་ཀྱི་མདོ། => D123
ལས་ཀྱི་སྒྲིབ་པ་རྣམ་པར་དག་པའི་མདོ། => D100
ལས་སྒྲིབ་རྒྱུན་གཅོད་ཀྱི་མདོ། => D123
སངས་རྒྱས་ཀྱི་སྡེ་སྣོད་ཚུལ་ཁྲིམས་འཆལ་པ་ཚར་གཅོད་པའི་མདོ། => D100
རྔ་བོ་ཆེ་ཆེན་པོའི་ལེའུའི་མདོ། => D109
སུམ་ཅུ་རྩ་གསུམ་པའི་ལེའུའི་མདོ། => D223
ལྷག་བསམ་བརྟན་པའི་ལེའུའི་མདོ། => D130
འཕགས་པ་གསུམ་ལ་སྐྱབས་སུ་འགྲོ་བའི་མདོ། => D100
སྲིད་པ་འཕོ་བའི་མདོ། => D125
རྣམ་པར་འཐག་པ་ཐམས་ཅད་བསྡུས་པའི་མདོ། => D100
སངས་རྒྱས་བགྲོ་བའི་མདོ། => D100
དེ་བཞིན་གཤེགས་པ་བགྲོ་བ་ཞེས་བྱ་བའི་མདོ། => D100
དེ་བཞིན་གཤེགས་པའི་དཔལ་གྱི་དམ་ཚིག་གི་མདོ། => D102
དཀོན་མཆོག་སྤྲིན་གྱི་མདོ། => D117


སྒྲོལ་མ་ལ་ཕྱག་འཚལ་བ་ཉི་ཤུ་གཅིག་གིས་བསྟོད་པ། => D438
ཕུར་པ་རྩ་བའི་རྒྱུད་ཀྱི་དུམ་བུ། => D105
ནག་པོ་ཆེན་པོའི་རྒྱུད། => D107
ནམ་མཁའ་དང་མཉམ་པའི་རྒྱུད། => D366
གསང་བ་འདུས་པ། => D442
དཔལ་གསང་བ་འདུས་པ་ཞེས་བྱ་བ་རྒྱུད་ཀྱི་རྒྱལ་པོ་ཆེན་པོ། => D100
རྒྱུད་ཕྱི་མ། => D363
དགོངས་པ་ལུང་བསྟན་པའི་རྒྱུད། => D100
རྡོ་རྗེ་འཕྲེང་བའི་རྒྱུད། => D370
ལྷ་མོ་བཞིས་ཡོངས་ཞུས། => D446
ཡེ་ཤེས་རྡོ་རྗེ་ཀུན་ལས་བཏུས་པའི་རྒྱུད། => D100
ཕྲིན་ལས་ཐམས་ཅད་འབྱུང་བའི་སྒྲོལ་མ་འདུས་པ་དོན་དམ་པ་ཞེས་བྱ་བ་རྣལ་འབྱོར་མའི་རྒྱུད་ཀྱི་རྒྱལ་པོ། => D100
རྡོ་རྗེ་སྙིང་པོ་རྒྱན་གྱི་རྒྱུད། => D362
གཉིས་སུ་མེད་པ་མཉམ་པ་ཉིད་རྣམ་པར་རྒྱལ་བའི་རྟོག་པའི་རྒྱལ་པོ། => D100
རྡོ་རྗེ་དཔལ་མཆོག་ཆེན་པོ་དང་པོ། => D111
ཕྱག་རྡོར་དྲག་པོ་གསུམ་འདུལ་གྱི་རྒྱུད། => D362
གསང་བའི་རྒྱུད་རྣམས་ཀྱི་རྣམ་པར་འབྱེད་པ་དྲག་པོ་གསུམ་འདུལ། => D100
ཕྱག་རྡོར་རྡོ་རྗེ་བེ་ཅོན་གྱི་རྒྱུད། => D370
ཕྱག་རྡོར་ཁམས་གསུམ་རྣམ་པར་རྒྱལ་བའི་རྒྱུད། => D142
རྡོ་རྗེ་གཏུམ་པོ་ཐུགས་གསང་བའི་རྒྱུད། => D108
རྡོ་རྗེ་གཏུམ་པོ་ཐུགས་གསང་བའི་རྒྱུད་ཕྱི་མ། => D104
རྡོ་རྗེ་གཏུམ་པོ་ཐུགས་གསང་བའི་རྒྱུད་ཕྱི་མའི་ཕྱི་མ། => D102
རྨ

སྣང་མཐའ་རྗེས་སུ་དྲན་པ། => D279
ཡོན་ཏན་བསྔགས་པ་དཔག་ཏུ་མེད་པའི་གཟུངས། => D101
བདེ་ལྡན་གྱི་སྙིང་པོ། => D412
སྤྱན་རས་གཟིགས་ཀྱི་རྩ་རྒྱུད་པདྨ་དྲ་བ། => D104
དོན་ཡོད་ཞགས་པའི་སྙིང་པོའི་མདོ། => D100
དོན་ཡོད་ཞགས་པའི་སྙིང་པོའི་གཟུངས། => D100
སྲེད་མེད་ཀྱི་བུས་ཞུས་པ་འཕགས་པ་སྒྱུ་མ་ཆེན་མོ་རྣམ་པར་རྒྱལ་བ་ཐོབ་པར་བྱེད་པ་ཞེས་བྱ་བའི་གཟུངས། => D100
ཀླུས་བྱིན་པའི་གཟུངས་ཀླུའི་སྙིང་པོ། => D100
དོན་ཡོད་ཞགས་པའི་ཆོ་ག་ཞིབ་མོའི་རྒྱལ་པོ། => D100
དོན་ཡོད་ཞགས་པའི་ཕར་ཕྱིན་དྲུག་ཡོངས་རྫོགས་ཀྱི་གཟུངས། => D100
ས་བཅུ་པའི་གཟུངས། => D139
དོན་ཡོད་ཞགས་པའི་རྟོག་པའི་རྒྱལ་པོའི་ཆོ་ག => D100
སྤྱན་རས་གཟིགས་ལག་སྟོང་མིག་སྟོང་གི་ཆོ་ག་ཞིབ་མོ། => D528
སྤྱན་རས་གཟིགས་ཕྱག་སྟོང་སྤྱན་སྟོང་གཟུངས། => D634
སྤྱན་རས་གཟིགས་ཡིད་བཞིན་འཁོར་ལོའི་གཟུངས། => D195
སྤྱན་རས་གཟིགས་ཞལ་བཅུ་གཅིག་པའི་གཟུངས། => D143
སྤྱན་རས་གཟིགས་ཞལ་བཅུ་གཅིག་པའི་རིགས་སྔགས་སྙིང་པོའི་གཟུངས། => D100
སྤྱན་རས་གཟིགས་ཀྱི་སྙིང་པོ། => D104
སྤྱན་རས་གཟིགས་ཀྱི་གཟུངས། => D102
ནཱི་ལ་ཀཎྛའི་གཟུངས། => D697
སྙིང་རྗེ་མི་བཤོལ་བའི་གཟུངས། => D304
ཀུན་ཏུ་བཟང་པོའི་གཟུངས། => D139
འཇིག་རྟེན་དབང་ཕྱུག་གི་རྟོག་

རི་ཁྲོད་ལོ་མ་གྱོན་པའི་གཟུངས། => D538
ཆོས་ཐམས་ཅད་ཀྱི་ཡུམ་ཞེས་བྱ་བའི་གཟུངས། => D100
འཕགས་མ་གཟའ་རྣམས་ཀྱི་ཡུམ་གྱི་གཟུངས། => D102
སྒྲོལ་མ་རང་གི་དམ་བཅས་པའི་གཟུངས། => D102
འཕགས་པ་རྒྱལ་བ་ཅན་གྱིས་གཟུངས། => D100
འཕགས་པ་དཔལ་ཆེན་མོའི་མདོ། => D100
དཔལ་ལྷ་མོའི་མཚན་བཅུ་གཉིས་པ། => D193
ནོར་གྱི་རྒྱུན་གྱི་གཟུངས། => D135
ལས་ཀྱི་སྒྲིབ་པ་ཐམས་ཅད་རྣམ་པར་སྦྱོང་བའི་གཟུངས། => D100
བར་གཅོད་ཐམས་ཅད་སེལ་བའི་གཟུངས་སྔགས། => D108
ནད་ཐམས་ཅད་རབ་ཏུ་ཞི་བར་བྱེད་པའི་གཟུངས། => D114
ནད་རབ་ཏུ་ཞི་བར་བྱེད་པའི་གཟུངས། => D129
རིམས་ནད་རབ་ཏུ་ཞི་བར་བྱེད་པའི་གཟུངས། => D129
མིག་ནད་ཞི་བར་བྱེད་པའི་མདོ། => D113
འབྲུམ་བུའི་ནད་ཞི་བར་བྱེད་པའི་གཟུངས། => D263
སྡང་བ་ཐམས་ཅད་རབ་ཏུ་ཞི་བར་བྱེད་པའི་གཟུངས། => D103
སྡིག་པ་ཐམས་ཅད་རབ་ཏུ་ཞི་བར་བྱེད་པའི་གཟུངས། => D100
སྡུག་བསྔལ་ཐམས་ཅད་རབ་ཏུ་ཞི་བར་བྱེད་པའི་གཟུངས། => D100
ཁྲོ་བ་ཞི་བར་བྱེད་པའི་གཟུངས། => D115
ཁྲོས་བཞི་བར་བྱེད་པའི་གཟུངས། => D103
ཚིག་བཙན་པའི་གཟུངས། => D143
བདག་སྲུང་བའི་གཟུངས། => D460a
དོན་ཐམས་ཅད་འགྲུབ་པའི་གཟུངས། => D143
ཤློ་ཀ་སྟོང་ལོབས་པའི་གཟུངས། => D356
ཤློ་ཀ་སུམ་སྟོང་ལོབས་པའི་གཟུངས། => D143
བ

In [93]:
result_yml = to_yaml(result)

In [94]:
Path('./text_id_tile.yml').write_text(result_yml)

112979

In [56]:
sim

0.5883484054145521